In [2]:
import pandas as pd
from pathlib import Path
import numpy as np


In [3]:
#Generate syllable frequency regressors for visual and bigram frequency regressors for auditory

In [4]:
master_table = pd.read_csv('/home/neel/Desktop/MOUS_hierarchical-representations/mous_words_syllable_bigram_frequencies.csv')
source = Path('/media/neel/MOUS/MOUS/MOUS/SynologyDrive/source')

In [5]:
master_table = master_table[['Word', 'Min_Freq_Count', 'Bigram Occurrence Counts']]
master_table.head()

,Word,Min_Freq_Count,Bigram Occurrence Counts
0,toen,177.0,"[2236591.0, 837795.0, 8542439.0]"
1,de,208410.0,[2559661.0]
2,barkeeper,1201.0,"[584273.0, 1754491.0, 2174749.0, 3143026.0, 94..."
3,die,441857.0,"[1050254.0, 2076673.0]"
4,irritante,8.0,"[2884214.0, 200692.0, 1787712.0, 333345.0, 892..."


In [6]:
#1. Syllable Frequency tables for visual modality

In [7]:

for subject in source.iterdir():
    if subject.name.startswith('sub-V') and subject.is_dir():
        transcription = pd.read_csv(subject / 'func' / (f"{subject.name}_transcription.csv"))
        #merge with master table on 'Word'
        transcription = transcription.merge(master_table, on='Word', how='left')
        transcription = transcription.rename(columns={'Min_Freq_Count': 'Log Min Syllable Frequency'})
        transcription['Log Min Syllable Frequency'] = transcription['Log Min Syllable Frequency'].apply(lambda x: np.log10(x))
        transcription = transcription[['Onset','Word', 'Log Min Syllable Frequency']]
        transcription.to_csv(subject / 'func' / (f"{subject.name}_syllable_frequency.csv"), index=False)
        


In [8]:
#2. Bigram Frequency tables for auditory modality

import ast  # safely parse string literals like "[123.0, 456.0]"

def compute_logmin_bigram(x):
    try:
        if isinstance(x, str):
            x = ast.literal_eval(x)  # safely convert string to list
        if isinstance(x, (list, tuple)):
            min_val = min(x)
        elif pd.notnull(x):
            min_val = float(x)
        else:
            return np.nan
        return np.log10(min_val) if min_val > 0 else np.nan
    except:
        return np.nan



In [9]:
for subject in source.iterdir():
    if subject.name.startswith('sub-A') and subject.is_dir():
        transcription = pd.read_csv(subject / 'func' / (f"{subject.name}_transcription.csv"))
        transcription = transcription.rename(columns={'Transcription': 'Word'})
        #merge with master table on 'Word'
        transcription = transcription.merge(master_table, on='Word', how='left')
        transcription['LogMin_Bigram_Frequency'] = transcription['Bigram Occurrence Counts'].apply(compute_logmin_bigram)
        transcription = transcription[['AlignOnset', 'Word', 'LogMin_Bigram_Frequency']]
        transcription.to_csv(subject / 'func' / (f"{subject.name}_bigram_frequency.csv"), index=False)


In [10]:
transcription

,AlignOnset,Word,LogMin_Bigram_Frequency
0,10.244,keek,5.882011
1,10.914,in,6.354996
2,11.434,die,6.021294
3,11.974,de,6.408182
4,12.464,verbaasd,4.723850
...,...,...,...
1375,1280.992,het,5.915397
1376,1281.082,nieuwe,4.995179
1377,1281.552,en,6.931582
1378,1281.782,eeuwige,4.995179
